# Supervised VAE for FRI and FRII Source Generation

## Introduction
### The Variational Auto Encoder
Auto Encoders are neural networks that consist of two networks an Encoder that reduced the higher dimensional data $x^{(i)}$ where $x^{(i)} \in R^{D}$ to a lower dimensional space $y^{(i)}$ where $y^{(i)} \in R^{d}$ and where $d < D$. The decoder takes the reduced vector $y^{(i)}$ back to higher dimensional space $\hat{x}^{(i)}$ where $\hat{x}^{(i)} \in R^{D}$. 

The neural network is then trained so as to minimize the difference between the output of the decoder and the input of the encoder.

The Variational Auto Encoder is an improved version of the typical Auto Encoder. Instead of outputting a conventional vectorat the Encoder, the latter will output two sets of vectors which are gaussian paramters. The VAE makes use of Variational Inference to optimize towards the solution.

### Mathematical Background


In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt


import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as dset
from torchvision.utils import save_image

from IPython.display import clear_output

import pyro
import pyro.distributions as dist
from pyro.infer import SVI, Trace_ELBO, TraceEnum_ELBO, config_enumerate
from pyro.optim import Adam,Adagrad

from PIL import Image 

from utils.MiraBest import MiraBest

from utils.FRDEEP import FRDEEPF



import collections

In [4]:
import network_configurations.neural_net_conf_0_0 as network #change this here to change configuration

In [5]:
from utils.classifier_fr import classification_procedure

In [6]:
pyro.enable_validation(True)
pyro.distributions.enable_validation(False)
pyro.set_rng_seed(0)

In [7]:
smoke_test = 'CI' in os.environ

In [18]:
def dataloader_first():
    transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize([0.5],[0.5])])

    trainset = MiraBest(root='./FIRST_data', train=True, download=True, transform=transform)  
    trainloader = torch.utils.data.DataLoader(trainset, shuffle=True, num_workers=2, batch_size=len(trainset))
    
    classes = ('FRI', 'FRII') #First class if FR1 and second class is FR2
    
    array_train= next(iter(trainloader))[0].numpy() # Training Datasets is loaded in numpy array
    array_label= next(iter(trainloader))[1].numpy()
    
    augmented_data=np.zeros((len(array_train)*36,1,100,100))
    
    augmented_data_label = np.zeros((len(array_train)*36,1))
    
    count=0
    
    for j in range(0,len(array_train)):
        image_object=Image.fromarray(array_train[j,0,:,:])
        for i in range(0,36):
            rotated=image_object.rotate(i*10)
            imgarr = np.array(rotated)
            temp_img_array=imgarr[25:125,25:125]
            augmented_data[count,0,:,:]=temp_img_array
            augmented_data_label[count,:]=array_label[j]
            count+=1
    augmented_data=(augmented_data-np.min(augmented_data))/(np.max(augmented_data)-np.min(augmented_data))
    
    X=augmented_data
    Y=augmented_data_label
    
    X_random_mix=np.take(X,np.random.RandomState(seed=42).permutation(X.shape[0]),axis=0,out=X)
    Y_random_mix=np.take(Y,np.random.RandomState(seed=42).permutation(Y.shape[0]),axis=0,out=Y)
    
    tensor_x = torch.stack([torch.Tensor(i) for i in X_random_mix])
    tensor_y = torch.stack([torch.Tensor(i) for i in Y_random_mix])
    
    first_augmented_dataset = torch.utils.data.TensorDataset(tensor_x,tensor_y)
    
    first_dataloader = torch.utils.data.DataLoader(first_augmented_dataset,batch_size=100, shuffle=True) # create your dataloader
    
    #--------------------------------------Add Section for Test data------------------------------------
    
    # Cropping of the Testing Images to 100 by 100 pixels
    
    
    testset = MiraBest(root='./FIRST_data', train=False, download=True, transform=transform) 
    
    testloader = torch.utils.data.DataLoader(testset, shuffle=True, num_workers=2, batch_size=len(testset))

    array_test= next(iter(testloader))[0].numpy()
    
    test_labels = next(iter(testloader))[1].numpy()
    
    test_data_reduced=np.zeros((len(array_test),1,100,100))
    test_data_label = np.zeros((len(array_test),1))
    
    for k in range (0,len(array_test)):
        test_data_reduced[k][0][:][:] = array_test[k][0][25:125,25:125]
        test_data_label[k,:]=test_labels[k]
    
    test_data_reduced=(test_data_reduced-np.min(test_data_reduced))/(np.max(test_data_reduced)-np.min(test_data_reduced))
    
    
    
    tensor_test = torch.stack([torch.Tensor(i) for i in test_data_reduced])
    tensor_test_label = torch.stack([torch.Tensor(i) for i in test_data_label])
    
    first_augmented_dataset_test = torch.utils.data.TensorDataset(tensor_test,tensor_test_label) # create your datset
    
    first_dataloader_test = torch.utils.data.DataLoader(first_augmented_dataset_test,batch_size=50, shuffle=True) # create your dataloader
    
    return first_dataloader,first_dataloader_test

#

In [40]:
class VAE(nn.Module):
    # by default our latent space is 50-dimensional
    # and we use 400 hidden units
    def __init__(self, x_dim=10000, y_dim=2, h_dim1=500, z_dim=2, use_cuda=True):
        super(VAE, self).__init__()
    
        # create the encoder and decoder networks
        # a split in the final layer's size is used for multiple outputs
        # and potentially applying separate activation functions on them
        # e.g. in this network the final output is of size [z_dim,z_dim]
        # to produce loc and scale, and apply different activations [None,Exp] on them
              
        self.encoder_z = network.EncoderZ(x_dim, y_dim, h_dim1, z_dim)
        
        self.decoder = network.Decoder(x_dim, y_dim, h_dim1, z_dim)

        if use_cuda:
            # calling cuda() here will put all the parameters of
            # the encoder and decoder networks into gpu memory
            self.cuda()
            
            
        self.use_cuda = use_cuda
        self.z_dim = z_dim
        self.output_size = y_dim
        
        
    # define the model p(x|z)p(z)
    def model(self, xs, ys):
        # register this pytorch module and all of its sub-modules with pyro
        pyro.module("ss_vae", self)
        batch_size = xs.size(0)

        # inform Pyro that the variables in the batch of xs, ys are conditionally independent
        with pyro.plate("data"):

            # sample the handwriting style from the constant prior distribution
            prior_loc = xs.new_zeros([batch_size, self.z_dim])
            prior_scale = xs.new_ones([batch_size, self.z_dim])
            zs = pyro.sample("z", dist.Normal(prior_loc, prior_scale).to_event(1))

            # if the label y (which digit to write) is supervised, sample from the
            # constant prior, otherwise, observe the value (i.e. score it against the constant prior)
            alpha_prior = xs.new_ones([batch_size, self.output_size]) / (1.0 * self.output_size)
            ys = pyro.sample("y", dist.OneHotCategorical(alpha_prior), obs=ys)
            
            # finally, score the image (x) using the handwriting style (z) and
            # the class label y (which digit to write) against the
            # parametrized distribution p(x|y,z) = bernoulli(decoder(y,z))
            # where `decoder` is a neural network
            loc = self.decoder.forward([zs, ys])
            pyro.sample("x", dist.Bernoulli(loc).to_event(1), obs=xs)
            
    def guide(self, xs, ys):
        with pyro.plate("data"):
           # if the class label (the digit) is not supervised, sample
           # (and score) the digit with the variational distribution
           # q(y|x) = categorical(alpha(x))
           
            #-------------------REMOVED THIS PART FOR THE CLASSIFIER ASSUME ALL DATA ARE LABELLED---------

           # sample (and score) the latent handwriting-style with the variational
           # distribution q(z|x,y) = normal(loc(x,y),scale(x,y))
           loc, scale = self.encoder_z.forward([xs, ys])
           pyro.sample("z", dist.Normal(loc, scale).to_event(1))

    # define a helper function for reconstructing images
    def reconstruct_img(self, xs, ys):
        # encode image x
        z_loc, z_scale = self.encoder_z.forward([xs,ys])
        # sample in latent space
        z = dist.Normal(z_loc, z_scale).sample()
        # decode the image (note we don't sample in image space)
        loc_img = self.decoder.forward([zs,ys])
        
        return loc_img


In [20]:
def train(svi, train_loader, use_cuda=True):
    # initialize loss accumulator
    epoch_loss = 0.
    # do a training epoch over each mini-batch x returned
    # by the data loader
    for x, y in train_loader:
        # if on GPU put mini-batch into CUDA memory
        if use_cuda:
            x = x.cuda()
        # do ELBO gradient and accumulate loss
        labels_y = torch.tensor(np.zeros((y.shape[0],2)))
        y_2=torch.Tensor.cpu(y.reshape(1,y.size()[0])[0]).numpy().astype(int)  
        labels_y=np.eye(2)[y_2]
        labels_y = torch.from_numpy(labels_y)   
         
        epoch_loss += svi.step(x.reshape(-1,10000),labels_y.cuda().float())

    # return epoch loss
    normalizer_train = len(train_loader.dataset)
    total_epoch_loss_train = epoch_loss / normalizer_train
    return total_epoch_loss_train

In [21]:
def evaluate(svi, test_loader, use_cuda=True):
    # initialize loss accumulator
    test_loss = 0.
    # compute the loss over the entire test set
    for x,y in test_loader:
        # if on GPU put mini-batch into CUDA memory
        if use_cuda:
            x = x.cuda()
        # compute ELBO estimate and accumulate loss
        test_loss += svi.evaluate_loss(x) #Data entry point <---------------------------------Data Entry Point
    normalizer_test = len(test_loader.dataset)
    total_epoch_loss_test = test_loss / normalizer_test
    return total_epoch_loss_test



In [22]:
def image_sample_plotter(epoch):
    with torch.no_grad():
        count = 0
        z_fr1 = z_fr2 = torch.randn(100, 2)
        labels_y1 = torch.tensor(np.zeros((100,2)))
        labels_y2 = torch.tensor(np.zeros((100,2)))
        for i in range (0,10):
            for j in range (0,10):
                z_fr1[count,0] = z_fr2[count,0] = np.random.uniform(-1,1)
                z_fr1[count,1] = z_fr2[count,1] = np.random.uniform(-1,1)
                labels_y1[count,0] = 1
                labels_y2[count,1] = 1
                count = count +1 
        
        sample1 = vae.decoder([z_fr1.cuda(),labels_y1.cuda().float()])
    
        save_image(sample1.view(100, 1, 100, 100), 'fr1_sample_2_z_space_' +str(epoch)+'.png',nrow=10)
    
        sample2 = vae.decoder([z_fr2.cuda(),labels_y2.cuda().float()])

        save_image(sample2.view(100, 1, 100, 100), 'fr2_sample_2_z_space_' +str(epoch)+'.png',nrow=10) 

In [23]:
def load_checkpoint(epoch):
    #print("loading model from ...")
    vae.load_state_dict(torch.load('/raid/scratch/davidb/1_DEVELOPMENT/VAE_FIRST/models/file_conf00x3_semi_supervised_vae_FRDEEP_epoch_'+str(epoch)))
   # print("loading optimizer states from ...")
    optimizer.load('/raid/scratch/davidb/1_DEVELOPMENT/VAE_FIRST/models/file_conf00x3_semi_supervised_vae_FRDEEP_epoch_'+str(epoch)+'_opt')
   # print("done loading model and optimizer states.")

In [24]:
def save_checkpoint(epoch):
    print("saving model to ...")
    torch.save(vae.state_dict(), '/raid/scratch/davidb/1_DEVELOPMENT/VAE_FIRST/models/file_conf00x3_semi_supervised_vae_FRDEEP_epoch_'+str(epoch))
    print("saving optimizer states...")
    optimizer.save('/raid/scratch/davidb/1_DEVELOPMENT/VAE_FIRST/models/file_conf00x3_semi_supervised_vae_FRDEEP_epoch_'+str(epoch)+'_opt')
    print("done saving model and optimizer checkpoints to disk.")


In [25]:
# Run options
LEARNING_RATE = 1.0e-3
USE_CUDA = True

# Run only for a single iteration for testing
NUM_EPOCHS = 20000
TEST_FREQUENCY = 5

In [26]:
results_array=np.zeros((1,2))

results_array_temp=np.zeros((1,2))

results_array_test=np.zeros((1,2))

results_array_temp_test=np.zeros((1,2))

In [27]:
train_loader,test_loader = dataloader_first()


Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# setup the VAE
pyro.clear_param_store()

net = classification_procedure()

vae = VAE(use_cuda=USE_CUDA)

# setup the optimizer
adagrad_params = {"lr": 0.001}
optimizer = Adagrad(adagrad_params)

svi = SVI(vae.model, vae.guide, optimizer, loss=Trace_ELBO())

train_elbo = []
test_elbo = []
# training loop
for epoch in range (0,2000):
 
    
    total_epoch_loss_train = train(svi, train_loader, use_cuda=USE_CUDA)
    
    
    train_loss = 0.
    for x, y in train_loader:
        x = x.cuda()
        y = y.cuda()
            # compute ELBO estimate and accumulate loss
        labels_y = torch.tensor(np.zeros((y.shape[0],2)))
        y_2=torch.Tensor.cpu(y.reshape(1,y.size()[0])[0]).numpy().astype(int)  
        labels_y=np.eye(2)[y_2]
        labels_y = torch.from_numpy(labels_y)
        
        train_loss += svi.evaluate_loss(x.reshape(-1,10000),labels_y.cuda().float()) 
            
            
    normalizer_train = len(train_loader.dataset)
    total_epoch_loss_train = train_loss / normalizer_train
        
    
    
    train_elbo.append(-total_epoch_loss_train)
    
    

    test_loss = 0.
        # compute the loss over the entire test set
    for x_test,y_test in test_loader:

            x_test = x_test.cuda()
            y_test = y_test.cuda()
            # compute ELBO estimate and accumulate loss
            labels_y_test = torch.tensor(np.zeros((y_test.shape[0],2)))
            y_test_2=torch.Tensor.cpu(y_test.reshape(1,y_test.size()[0])[0]).numpy().astype(int)  
            labels_y_test=np.eye(2)[y_test_2]
            labels_y_test = torch.from_numpy(labels_y_test)
        
            test_loss += svi.evaluate_loss(x_test.reshape(-1,10000),labels_y_test.cuda().float()) 
            
            
    normalizer_test = len(test_loader.dataset)
    total_epoch_loss_test = test_loss / normalizer_test
    
    z_fr1 = z_fr2 = torch.FloatTensor(100, 2).uniform_(-2, 2)
    labels_y1 = torch.tensor(np.zeros((100,2)))
    labels_y2 = torch.tensor(np.zeros((100,2)))
        
    labels_y1[:,1] = 0
    labels_y2[:,1] = 0

    labels_y1[:,0] = 0
    labels_y2[:,0] = 0

    labels_y1[:,1] = 2
    labels_y2[:,0] = 2

    sample_fr1 = vae.decoder([z_fr1.cuda(),labels_y1.cuda().float()])
    img1=sample_fr1.reshape(100,100,100).cpu().detach().numpy()
 

    sample_fr2 = vae.decoder([z_fr2.cuda(),labels_y2.cuda().float()])
    img2=sample_fr2.reshape(100,100,100).cpu().detach().numpy()

    classification_results = np.zeros((1,100))
    
    array_fr1 = np.zeros(100)
    array_fr2 = np.zeros(100)
    for i in range (0,100):
        img_tensor = torch.tensor(np.zeros((1,1,150,150)))
        img_tensor[0,0,25:125,25:125]=sample_fr2[i].reshape(100,100)
        outputs = net(img_tensor.float())
        _, predicted = torch.max(outputs, 1)

        array_fr2[i]= float(predicted[0].cpu().detach().numpy())

        img_tensor = torch.tensor(np.zeros((1,1,150,150)))
        img_tensor[0,0,25:125,25:125]=sample_fr1[i].reshape(100,100)
        outputs = net(img_tensor.float())
        _, predicted = torch.max(outputs, 1)
    
        array_fr1[i]= float(predicted[0].cpu().detach().numpy())   
    
    print('Train Loss: '+str(total_epoch_loss_train)+' Test Loss:'+str(total_epoch_loss_test)+' VAE Efficiency:'+str((collections.Counter(array_fr1)[1]+collections.Counter(array_fr2)[0])/200))
  

In [ ]:
z_fr1 = z_fr2 = torch.FloatTensor(100, 2).uniform_(-2, 2)
labels_y1 = torch.tensor(np.zeros((100,2)))
labels_y2 = torch.tensor(np.zeros((100,2)))
        
labels_y1[:,1] = 0
labels_y2[:,1] = 0

labels_y1[:,0] = 0
labels_y2[:,0] = 0

labels_y1[:,1] = 2
labels_y2[:,0] = 2

sample_fr1 = vae.decoder([z_fr1.cuda(),labels_y1.cuda().float()])
img1=sample_fr1.reshape(100,100,100).cpu().detach().numpy()
 

sample_fr2 = vae.decoder([z_fr2.cuda(),labels_y2.cuda().float()])
img2=sample_fr2.reshape(100,100,100).cpu().detach().numpy()

classification_results = np.zeros((1,100))




In [ ]:
for i in range (0,50):
    fig = plt.figure(figsize=(10,10))
    ax1 = fig.add_subplot(2,2,1)
    ax1.imshow(sample_fr1[i].reshape(100,100).cpu().detach().numpy())
    ax2 = fig.add_subplot(2,2,2)
    ax2.imshow(img2[i])
    

In [ ]:
import collections

In [ ]:
collections.Counter(array_fr2)

In [ ]:
num = 85
fig = plt.figure(figsize=(10,10))
ax1 = fig.add_subplot(2,2,1)
ax1.imshow(img1[num])
ax2 = fig.add_subplot(2,2,2)
ax2.imshow(img2[num])

In [ ]:
img_tensor = torch.tensor(np.zeros((1,1,150,150)))
img_tensor[0,0,25:125,25:125]==sample_fr2[i].reshape(100,100)
outputs = net(img_tensor.float())
_, predicted = torch.max(outputs, 1)
fig = plt.figure(figsize=(10,10))
ax1 = fig.add_subplot(2,2,1)
ax1.imshow(img1[i])
ax2 = fig.add_subplot(2,2,2)
ax2.imshow(img2[i])

In [ ]:
for i in range (0,50):
    fig = plt.figure(figsize=(10,10))
    ax1 = fig.add_subplot(2,2,1)
    ax1.imshow(sample_fr1[i].reshape(100,100).cpu().detach().numpy())
    ax2 = fig.add_subplot(2,2,2)
    ax2.imshow(img2[i])
    